<a href="https://colab.research.google.com/github/napstruck/DetectTheAudio/blob/main/Major_Project_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-federated librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import librosa
import os
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten


/usr/local/lib/python3.11/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.11/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = "/content/drive/MyDrive/audio_dataset.zip"

# Directory to extract the dataset
extract_path = "/content/dataset"

# Create the extraction directory
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

In [ ]:
def extract_features(audio, sample_rate=16000):
    """
    Extract multiple audio features from a waveform.
    Args:
        audio (tf.Tensor): Audio waveform (1D tensor).
        sample_rate (int): Sample rate of the audio.
    Returns:
        features (dict): Dictionary of extracted features.
    """
    audio = audio.numpy()  # Convert TensorFlow tensor to NumPy array
    features = {}

    # 1. MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    features['mfccs'] = mfccs.T  # Transpose to match time steps

    # 2. Chroma Features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    features['chroma'] = chroma.T

    # 3. Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    features['spectral_contrast'] = spectral_contrast.T

    # 4. Zero-Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
    features['zero_crossing_rate'] = zero_crossing_rate.T

    # 5. RMS Energy
    rms_energy = librosa.feature.rms(y=audio)
    features['rms_energy'] = rms_energy.T

    # 6. Mel Spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    features['mel_spectrogram'] = mel_spectrogram.T

    return features
